In [4]:
import pandas as pd
import gzip

# List of file paths
file_paths = [
    'home_rental_data/Ireland/reviews.csv.gz',
    'home_rental_data/Lisbon/reviews.csv.gz',
    'home_rental_data/Los Angeles/reviews.csv.gz',
    'home_rental_data/Malaga/calendar.csv',
    'home_rental_data/New Zeland/reviews.csv.gz',
    'home_rental_data/Rome/reviews.csv.gz'
]

# Initialize an empty dictionary to store the dataframes
dataframes = {}

# Loop through the file paths and read the files into dataframes
for file_path in file_paths:
    if file_path.endswith('.gz'):
        with gzip.open(file_path, 'rt') as file:
            df = pd.read_csv(file)
    else:
        df = pd.read_csv(file_path)

    # Extract the filename without the path
    filename = file_path.split('/')[-1]

    # Remove the file extension
    filename = filename.split('.')[0]

    # Store the dataframe in the dictionary with the filename as the key
    dataframes[filename] = df

# Now you can access the dataframes using their respective keys
ireland_reviews_df = dataframes['Ireland_reviews']
lisbon_reviews_df = dataframes['Lisbon_reviews']
los_angeles_reviews_df = dataframes['Los Angeles_reviews']
malaga_calendar_df = dataframes['Malaga_calendar']
new_zealand_reviews_df = dataframes['New Zeland_reviews']
rome_reviews_df = dataframes['Rome_reviews']

KeyError: 'Ireland_reviews'

In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.read_csv("home_rental_data/Amsterdam/listings.csv")

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2818,Quiet Garden View Room & Super Fast WiFi,3159,Daniel,NaN,Oostelijk Havengebied - Indische Buurt,52.36435,4.94358,Private room,49,3,305,2022-08-30,1.86,1,14,25,0363 5F3A 5684 6750 D14D
1,20168,Studio with private bathroom in the centre 1,59484,Alexander,NaN,Centrum-Oost,52.36407,4.89393,Private room,106,1,339,2020-04-09,2.22,2,0,0,0363 CBB3 2C10 0C2A 1E29
2,27886,"Romantic, stylish B&B houseboat in canal district",97647,Flip,NaN,Centrum-West,52.38761,4.89188,Private room,136,2,231,2022-04-24,1.78,1,121,8,0363 974D 4986 7411 88D8
3,28871,Comfortable double room,124245,Edwin,NaN,Centrum-West,52.36775,4.89092,Private room,75,2,428,2022-08-24,2.92,2,117,75,0363 607B EA74 0BD8 2F6F
4,29051,Comfortable single room,124245,Edwin,NaN,Centrum-Oost,52.36584,4.89111,Private room,55,2,582,2022-08-29,4.16,2,160,86,0363 607B EA74 0BD8 2F6F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6888,12690380,"Amsterdam, light big house 15 min from citycenter",15032494,Lennard,NaN,Buitenveldert - Zuidas,52.31623,4.86718,Entire home/apt,300,3,28,2022-08-20,0.36,1,0,2,NaN
6889,15005354,Perfect family home near Amsterdam,46437958,Gea,NaN,Buitenveldert - Zuidas,52.30863,4.86690,Entire home/apt,195,4,18,2022-07-26,0.26,1,0,1,NaN
6890,15171981,Child friendly house near Amsterdam - free par...,96286859,Ischa,NaN,Watergraafsmeer,52.33370,4.94073,Entire home/apt,449,1,21,2022-08-07,0.29,1,0,2,NaN
6891,17567812,"Modern, spacious family home, 5 bedrooms-Amste...",55072916,Manon,NaN,Buitenveldert - Zuidas,52.31309,4.84703,Entire home/apt,282,4,8,2019-08-05,0.12,1,0,0,NaN
